In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [13]:
drink_df = pd.read_csv('all_drinks.csv')
drink_df.sample(3)

,Unnamed: 0,strDrink,dateModified,idDrink,strAlcoholic,strCategory,strDrinkThumb,strGlass,strIBA,strIngredient1,strIngredient10,strIngredient11,strIngredient12,strIngredient13,strIngredient14,strIngredient15,strIngredient2,strIngredient3,strIngredient4,strIngredient5,strIngredient6,strIngredient7,strIngredient8,strIngredient9,strInstructions,strMeasure1,strMeasure10,strMeasure11,strMeasure12,strMeasure13,strMeasure14,strMeasure15,strMeasure2,strMeasure3,strMeasure4,strMeasure5,strMeasure6,strMeasure7,strMeasure8,strMeasure9,strVideo
218,218,Dark and Stormy,2017-09-02 18:55:05,17211,Alcoholic,Ordinary Drink,http://www.thecocktaildb.com/images/media/drin...,Highball glass,New Era Drinks,Dark Rum,NaN,NaN,NaN,NaN,NaN,NaN,Ginger Beer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In a highball glass filled with ice add 6cl da...,5 cl,NaN,NaN,NaN,NaN,NaN,NaN,10 cl,NaN,NaN,NaN,,,NaN,NaN,NaN
463,463,Sherry Flip,2016-11-10 23:27:08,12190,Alcoholic,Ordinary Drink,http://www.thecocktaildb.com/images/media/drin...,Whiskey sour glass,NaN,Sherry,NaN,NaN,NaN,NaN,NaN,NaN,Light cream,Powdered sugar,Egg,Nutmeg,NaN,NaN,NaN,NaN,Shake all ingredients (except nutmeg) with ice...,1 1/2 oz cream,\n,NaN,NaN,NaN,NaN,NaN,2 tsp,1 tsp,1 whole,\n,\n,\n,\n,\n,NaN
20,20,A Splash of Nash,2016-08-31 19:30:01,14564,Alcoholic,Shot,http://www.thecocktaildb.com/images/media/drin...,Highball glass,NaN,Cranberry juice,NaN,NaN,NaN,NaN,NaN,NaN,Soda water,Midori melon liqueur,Creme de Banane,NaN,NaN,NaN,NaN,NaN,Drop shot glass with banana & melon liquers in...,2 oz,NaN,NaN,NaN,NaN,NaN,NaN,2 oz,0.5 oz,0.5 oz,,,,,,NaN


In [14]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

In [15]:
str_vec = drink_df['strDrink'].str.lower()
MAX_NB_WORDS, MAX_SEQUENCE_LENGTH = 100, 40
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, char_level=True)
tokenizer.fit_on_texts(str_vec)
train_sequences = tokenizer.texts_to_sequences(str_vec)
train_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
char_index = tokenizer.word_index
print('Found %s unique tokens.' % len(char_index))
train_x = np.stack([to_categorical(x, num_classes=len(char_index)+1) for x in train_data],0)
print(train_x.shape)

Found 47 unique tokens.
(546, 40, 48)


In [16]:
def isempty(x):
    try:
        if x is None: 
            return True
        elif len(x)<1:
            return True
        else:
            return False
    except:
        # floating point nans
        return True
all_ingred = drink_df[[x for x in drink_df.columns 
                       if 'Ingredient' in x]].apply(lambda c_row: [v.lower() for k,v in c_row.items() if not isempty(v)],1)
all_ingred[0:3]

0                              [creme de cacao, vodka]
1    [absolut kurant, grand marnier, chambord raspb...
2                                     [lager, tequila]
dtype: object

In [18]:
from sklearn.preprocessing import LabelEncoder

ingred_label = LabelEncoder()
ingred_label.fit(list(chain(*all_ingred.values)))
print('Found', len(ingred_label.classes_), 'unique ingredients, ', ingred_label.classes_[0:3])

Found 308 unique ingredients,  ['151 proof rum' '7-up' 'absinthe']


In [19]:
y_vec = np.stack(all_ingred.map(lambda x: np.sum(to_categorical(ingred_label.transform(x), 
                                        num_classes=len(ingred_label.classes_)),0)),0).clip(0,1)

In [20]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
train_idx, test_idx = train_test_split(range(y_vec.shape[0]), 
                                                    random_state = 12345,
                                                   train_size = 0.7)

In [21]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Masking
from keras.optimizers import Adam
# model based loosely on https://github.com/fchollet/keras/blob/master/examples/imdb_lstm.py
simple_sequence_model = Sequential()
simple_sequence_model.add(Masking(0, input_shape = (None,)))
simple_sequence_model.add(Embedding(len(char_index)+1, 32))
simple_sequence_model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
simple_sequence_model.add(Dense(y_vec.shape[1], 
                                activation = 'sigmoid'))
simple_sequence_model.compile(loss = 'binary_crossentropy', # categorical and mae don't work well here
                              optimizer = Adam(lr = 5e-4, decay = 1e-6), 
                             metrics = ['mae'])
simple_sequence_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 32)          1536      
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               82432     
_________________________________________________________________
dense_1 (Dense)              (None, 308)               39732     
Total params: 123,700
Trainable params: 123,700
Non-trainable params: 0
_________________________________________________________________


In [22]:
simple_sequence_model.fit(train_data[train_idx], y_vec[train_idx], epochs=10,
                          batch_size = 32,
                         validation_data = (train_data[test_idx], y_vec[test_idx]), 
                          verbose = 1)
pred_vec = simple_sequence_model.predict(train_data[test_idx])

print('Mean Error %2.2f%%' % (100*mean_absolute_error(y_vec[test_idx], pred_vec)))

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 382 samples, validate on 164 samples
Epoch 1/10
382/382 [==============================] - ETA: 4s - loss: 0.6932 - mae: 0.500 - ETA: 1s - loss: 0.6928 - mae: 0.499 - ETA: 0s - loss: 0.6923 - mae: 0.499 - ETA: 0s - loss: 0.6918 - mae: 0.499 - ETA: 0s - loss: 0.6912 - mae: 0.499 - ETA: 0s - loss: 0.6904 - mae: 0.498 - 1s 2ms/step - loss: 0.6900 - mae: 0.4984 - val_loss: 0.6827 - val_mae: 0.4947
Epoch 2/10
382/382 [==============================] - ETA: 0s - loss: 0.6824 - mae: 0.494 - ETA: 0s - loss: 0.6796 - mae: 0.493 - ETA: 0s - loss: 0.6711 - mae: 0.488 - ETA: 0s - loss: 0.6603 - mae: 0.482 - ETA: 0s - loss: 0.6415 - mae: 0.472 - 0s 927us/step - loss: 0.6183 - mae: 0.4585 - val_loss: 0.4440 - val_mae: 0.3516
Epoch 3/10
382/382 [==============================] - ETA: 0s - loss: 0.4494 - mae: 0.355 - ETA: 0s - loss: 0.4127 - mae: 0.330 - ETA: 0s - loss: 0.3787 - mae: 0.306 - ETA: 0s - loss: 0.3580 - mae: 0.290 - 0s 864us/step - loss: 0.3297 - mae: 0.2696 - val_loss: 0.2137 - 

In [26]:
print('Input Name:', drink_df['strDrink'].values[test_idx[0]])
print('Real Ingredients', all_ingred.values[test_idx[0]])

proc_pred = lambda out_pred: sorted([(ingred_label.inverse_transform(idx), out_pred[idx])
                              for idx in np.where(out_pred>0)[0]], key = lambda x: -x[1])

print('Predicted Ingredients')
for _, (i,j) in zip(range(5), proc_pred(pred_vec[0])):
    print('%25s\t\t%2.2f%%' % (i,100*j))

Input Name: Kioki Coffee
Real Ingredients ['kahlua', 'brandy', 'coffee']
Predicted Ingredients


ValueError: bad input shape ()